In [2]:
import PyPDF2 #20250620
import glob,os
import fitz
from PyPDF2 import PdfReader, PdfWriter

page_images = []
page_text = []
username = 'john.tan'
path_pdf = r"C:/Users/"+username+r"/Downloads/esker_merged/" #esker_merged
pdf_files = [f for f in os.listdir(path_pdf) if (f.lower().endswith('.pdf'))] #[pdf_files[0]]

def find_text_in_pdf(path_pdf, file_pdf):
    """
    Removes blank pages and the first page if it's blank from a PDF file.
    Args:
        path_pdf (str): Path to the directory containing the PDF file.
        file_pdf (str): Name of the PDF file.
    Returns:
        None
    """
    with open(path_pdf + file_pdf, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        pdf_writer = PyPDF2.PdfWriter()

        # Check the first page
        if pdf_reader.pages:  # Ensure there are pages
            first_page = pdf_reader.pages[0]
            first_page_text = first_page.extract_text()
            if first_page_text.strip() == "":
                # First page is blank, skip it
                start_page = 1
            else:
                start_page = 0
        else:
            return # no pages to process

        # Process remaining pages
        for page_num in range(start_page, len(pdf_reader.pages)): #len(pdf_reader.pages)
                page = pdf_reader.pages[page_num]
                text = page.extract_text()
                if text.strip() != "":
                    #pdf_writer.add_page(page)
                    page_text.append(page_num)
                    #print(f"Page {page_num} has text: {text}")
    #print(len(page_text))
    #print(f"total pages: {len(pdf_reader.pages)}")
    return page_text


def find_images_in_pdf(path_pdf, file_pdf):
    pdf = fitz.Document(path_pdf + file_pdf)
    #page_images = []
    for i in range(len(pdf)):
        images = pdf.get_page_images(i)
        has_image = False if len(images) == 0 else True
        # print(f"{i}: {hasimage}: {images}")
        if has_image:
            page_images.append(i)
            #print(f"Page {i} has images: {images}")
    #print(f"total pages: {len(pdf)}")
    return page_images
    
        # Write the new PDF
        #with open(path_pdf + "no_blank_" + file_pdf, 'wb') as output_file:
                #pdf_writer.write(output_file)

def rename_pdf_files(src_directory):
    """
    Renames PDF files in the specified directory by replacing 'no_blank_' with ''.
    Args:
        src_directory (str): The directory path containing PDF files to rename.
    """
    # Get a list of PDF files in the directory
    pdf_files = [file for file in os.listdir(src_directory) if file.endswith('.pdf')] #and 'no_blank_' in file
    i=0
    # Rename PDF files by replacing 'no_blank_' with ''
    for file in pdf_files:
        new_filename = file.replace('no_blank_', '')
        if os.path.exists(os.path.join(src_directory, new_filename)):
            print(f"A file with the name '{new_filename}' already exists. Skipping...")
        else:
            os.rename(os.path.join(src_directory, file), os.path.join(src_directory, new_filename))
            #print(f"Renamed '{file}' to '{new_filename}'")
            i +=1
    return i

def rename_pdf_files_no_blank(src_directory):
    """
    Renames PDF files in the specified directory by replacing 'no_blank_' with ''.
    Args:
        src_directory (str): The directory path containing PDF files to rename.
    """
    # Get a list of PDF files in the directory
    pdf_files = [file for file in os.listdir(src_directory) if file.endswith('.pdf') and 'no_blank_' in file]
    i=0
    # Rename PDF files by replacing 'no_blank_' with ''
    for file in pdf_files:
        try:
            new_filename = file.replace('no_blank_', '')
        except Exception as e:
            print(e)
        try:
            new_filename = file.replace('_merged', '')
        except Exception as e:
            print(e)
        if os.path.exists(os.path.join(src_directory, new_filename)):
            print(f"A file with the name '{new_filename}' already exists. Skipping...")
        else:
            os.rename(os.path.join(src_directory, file), os.path.join(src_directory, new_filename))
            #print(f"Renamed '{file}' to '{new_filename}'")
            i +=1
    return i

def remove_first_page_from_pdfs(src_dir):
    pdf_files = [os.path.join(src_dir, file) for file in os.listdir(src_dir) if file.endswith('.pdf')]
    processed_count = 0

    for file_path in pdf_files:
        try:
            reader = PdfReader(file_path)
            if len(reader.pages) < 1:
                print(f"Skipping {file_path} as it has no pages.")
                continue

            # Open the PDF with fitz to extract text from the first page
            doc = fitz.open(file_path)
            first_page_text = doc[0].get_text()
            doc.close()

            # Check if the first page is blank
            if not first_page_text.strip():
                writer = PdfWriter()

                # Add all pages except the first one
                for page_num in range(1, len(reader.pages)):
                    writer.add_page(reader.pages[page_num])

                # Save the modified PDF
                with open(file_path, 'wb') as new_file:
                    writer.write(new_file)

                print(f"Removed first page from {file_path} as it was blank.")
            else:
                print(f"First page of {file_path} is not blank, skipping.")

            processed_count += 1
        except Exception as e:
            print(f"Error processing {file_path}: {str(e)}")

    return processed_count

src_directory = 'C:/Users/john.tan/Downloads/esker_merged/' #esker_merged
def main():
    for pdf_file in pdf_files:
        try:
            find_text_in_pdf(path_pdf, pdf_file)
        except Exception as e:
            print(e)
            break
        try:
            find_images_in_pdf(path_pdf, pdf_file)
        except Exception as e:
            print(e)
            break
        list_pages_text_images = list(set(page_images + page_text))
        with open(path_pdf + pdf_file, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            pdf_writer = PyPDF2.PdfWriter()

            for page_num in list_pages_text_images:
                try:

                    page = pdf_reader.pages[page_num]
                    pdf_writer.add_page(page)
                except Exception as e:  
                    print(e)
                    break
            with open(path_pdf + "" + pdf_file, 'wb') as output_file:
                pdf_writer.write(output_file) #write the new pdf
    remove_first_page_from_pdfs(src_directory)
    #rename_pdf_files(src_directory)
   

if __name__ == "__main__":
    main()

"""
print(f"page_has_text: {page_text}")
print(f"total_page_has_text: {len(page_text)}")
#print(f"total_pages: {len(pdf_reader.pages)}")
print(f"page_has_image: {page_images}")
print(f"total_page_has_image: {len(page_images)}")
"""

#run

sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out o

unknown widths : 
[0, IndirectObject(63, 0, 2391322479504)]
unknown widths : 
[0, IndirectObject(69, 0, 2391322479504)]
unknown widths : 
[0, IndirectObject(75, 0, 2391322479504)]
unknown widths : 
[0, IndirectObject(81, 0, 2391322479504)]
unknown widths : 
[0, IndirectObject(87, 0, 2391322479504)]
unknown widths : 
[0, IndirectObject(100, 0, 2391322479504)]
unknown widths : 
[0, IndirectObject(106, 0, 2391322479504)]
unknown widths : 
[0, IndirectObject(112, 0, 2391322479504)]
unknown widths : 
[0, IndirectObject(118, 0, 2391322479504)]
unknown widths : 
[0, IndirectObject(124, 0, 2391322479504)]
unknown widths : 
[0, IndirectObject(130, 0, 2391322479504)]
unknown widths : 
[0, IndirectObject(136, 0, 2391322479504)]
unknown widths : 
[0, IndirectObject(142, 0, 2391322479504)]
unknown widths : 
[0, IndirectObject(148, 0, 2391322479504)]
unknown widths : 
[0, IndirectObject(154, 0, 2391322479504)]
unknown widths : 
[0, IndirectObject(160, 0, 2391322479504)]
unknown widths : 
[0, Indirec

sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out of range
sequence index out o

'\nprint(f"page_has_text: {page_text}")\nprint(f"total_page_has_text: {len(page_text)}")\n#print(f"total_pages: {len(pdf_reader.pages)}")\nprint(f"page_has_image: {page_images}")\nprint(f"total_page_has_image: {len(page_images)}")\n'

In [2]:
import os
from PyPDF2 import PdfReader, PdfWriter

def process_pdfs(src_dir):
    pdf_files = [os.path.join(src_dir, file) for file in os.listdir(src_dir) if file.endswith('.pdf')]
    processed_count = 0

    for file_path in pdf_files:
        try:
            reader = PdfReader(file_path)
            writer = PdfWriter()

            # Add all pages except the first one
            for page_num in range(1, len(reader.pages)):
                writer.add_page(reader.pages[page_num])

            # Save the modified PDF
            with open(file_path, 'wb') as new_file:
                writer.write(new_file)

            processed_count += 1
        except Exception as e:
            print(f"Error processing {file_path}: {str(e)}")

    return processed_count

# Example usage:
#src_dir = 'C:/Users/john.tan/Downloads/esker_merged/'
src_dir = 'C:/Users/john.tan/Documents/AP/20250717'
processed_count = process_pdfs(src_dir)
print(f"Processed {processed_count} PDF files.")


Processed 90 PDF files.


In [ ]:
import os #20250703 good!
from PyPDF2 import PdfReader, PdfWriter

def remove_first_page_from_pdfs(src_dir):
    """
    Remove the first page from all PDF files in src_dir and save them back.    
    Args:
        src_dir (str): Source directory containing PDF files        
    Returns:
        int: Number of files that had their first page removed
    """
    if not os.path.exists(src_dir):
        raise ValueError(f"Directory {src_dir} does not exist")    
    processed_count = 0
    
    # Get all PDF files (both .pdf and .PDF extensions)
    pdf_files = []
    for file in os.listdir(src_dir):
        if file.lower().endswith('.pdf'):
            pdf_files.append(file)    
    for pdf_file in pdf_files:
        file_path = os.path.join(src_dir, pdf_file)
        
        try:
            # Read the PDF
            reader = PdfReader(file_path)
            
            # Skip files with only one page or empty files
            if len(reader.pages) <= 1:
                continue
            # Create a new PDF writer with all pages except the first
            writer = PdfWriter()
            for page_num in range(1, len(reader.pages)):
                writer.add_page(reader.pages[page_num])
            
            # Save the modified PDF back to the same location
            with open(file_path, 'wb') as output_file:
                writer.write(output_file)
            
            processed_count += 1            
        except Exception as e:
            print(f"Error processing {pdf_file}: {str(e)}")
            continue
    
    return processed_count

src_dir = "C:/Users/john.tan/Downloads/esker_merged/"
remove_first_page_from_pdfs(src_dir)

10